# Minimally Sufficient Pandas with Ted Petrou

* Author of Pandas Cookbook

* Founder of Dunder Data

# Collect Data

* Who knows that Pandas refers to a Python library as well as an east-Asian bear?

* Have you used Pandas before?

* Have you used Pandas in production before?

# Do these apply to you
* Don't know the difference between `[], .iloc, .loc, .ix, .at, .iat`
* Use `reset_index` frequently because you have no idea how to deal with MultiIndexes
* Use for-loops frequently
* Use `apply` frequently
* Struggle with Pandas, and find yourself wishing it was easy as R

# Pandas Quiz #1
# How do you select the food column?

In [1]:
import pandas as pd
df = pd.read_csv('data/sample_data.csv', index_col=0)
df

,state,color,food,age,height,score
Jane,NY,blue,Steak,30,165,4.6
Niko,TX,green,Lamb,2,70,8.3
Aaron,FL,red,Mango,12,120,9.0
Penelope,AL,white,Apple,4,80,3.3
Dean,AK,gray,Cheese,32,180,1.8
Christina,TX,black,Melon,33,172,9.5
Cornelia,TX,red,Beans,69,150,2.2


In [6]:
df['food'].max()

'Steak'

In [2]:
df['food'].

Jane          Steak
Niko           Lamb
Aaron         Mango
Penelope      Apple
Dean         Cheese
Christina     Melon
Cornelia      Beans
Name: food, dtype: object

# Pandas Quiz #2
### How do you select the row just for Penelope?

In [10]:
df.loc[['Penelope']]

,state,color,food,age,height,score
Penelope,AL,white,Apple,4,80,3.3


In [9]:
df

,state,color,food,age,height,score
Jane,NY,blue,Steak,30,165,4.6
Niko,TX,green,Lamb,2,70,8.3
Aaron,FL,red,Mango,12,120,9.0
Penelope,AL,white,Apple,4,80,3.3
Dean,AK,gray,Cheese,32,180,1.8
Christina,TX,black,Melon,33,172,9.5
Cornelia,TX,red,Beans,69,150,2.2


# Pandas Quiz #3
### How would you select the food and age columns for everyone over the age of 30?

In [12]:
df.loc[df["age"]>30,["food","age"]]

,food,age
Dean,Cheese,32
Christina,Melon,33
Cornelia,Beans,69


In [11]:
df

,state,color,food,age,height,score
Jane,NY,blue,Steak,30,165,4.6
Niko,TX,green,Lamb,2,70,8.3
Aaron,FL,red,Mango,12,120,9.0
Penelope,AL,white,Apple,4,80,3.3
Dean,AK,gray,Cheese,32,180,1.8
Christina,TX,black,Melon,33,172,9.5
Cornelia,TX,red,Beans,69,150,2.2


# Minimally Sufficient Pandas

* There are multiple ways to accomplish most tasks

* Often, there is not an obvious way to do things

* A small subset of the library covers nearly all of the possible tasks

* Knowing many obscure Pandas tricks is not helpful

* Developing a standard Pandas usage guide can be helpful

* Pandas can be written in a very implicit way. Be as explicit as possible.

* Ask yourself whether method B gives you more functionality than method A

* Pandas is difficult to use in production - striving for consistency and simplicity can make a big difference

* There are an incredible amount of issues/bugs and using a minimally sufficient subset of Pandas can help avoid landing on a bug

# Simple Guidelines

* Use only bracket notation and never dot notation to select single columns
    * Columns with spaces do not work
    * Column names that collide with methods do not work

* Only use string names for columns

* Avoid chained indexing, especially when assigning new values to subsets of data
    * Do not do this: `df[df['col1'] > 10]['col2'] = 10`

* Never use `.ix` for subset selection. It is deprecated.
* No reason to use `.at` and `.iat`

* Use bracket notation instead of the `query` method to do boolean selection

* Use the arithmetic and comparison operators instead of their counterpart methods (`add`, `gt`, etc...)

* Use DataFrame/Series methods when they exist
    * Avoid built-in `Python` functions
    * Avoid the `apply` method when possible

* Do not store complex data types in DataFrame/Series values - i.e. no lists, Series, or DataFrames within DataFrames/Series

* Decide on a syntax for grouping (especially when aggregating)
    * `df.groupby(['grouping', 'columns']).agg({'aggregating column': 'aggregating func'})`
    * `df.groupby(['grouping', 'columns'])['aggregating column'].aggregating_func()`

* Have a standard way of handling a multi-level Index
    * Should you reset to single level? 
    * Should you reset and rename multi-level column indexes?

* Be very careful when calling `apply` on a `groupby` - this is the slowest operation in Pandas
    * Pre-calculate anything that is independent of the group

* `melt/pivot` vs `stack/unstack` - They both do the same thing

# Chained Indexing
Occurs when consecutive subset selection. If you see back to back brackets (`][`), you have done chained indexing. 

In [13]:
df[['color', 'food', 'state']][['color', 'food']]

,color,food
Jane,blue,Steak
Niko,green,Lamb
Aaron,red,Mango
Penelope,white,Apple
Dean,gray,Cheese
Christina,black,Melon
Cornelia,red,Beans


In [14]:
# using a single indexer
df.loc[df['age'] > 30, ['color', 'food']]

,color,food
Dean,gray,Cheese
Christina,black,Melon
Cornelia,red,Beans


### Helpful to break apart row and column selection

In [ ]:
rs = df['age'] > 30
cs = ['color', 'food']
df.loc[rs, cs]

# Two common scenarios when assigning subsets of data
1. You want to make an assignment to a particular subset of your DataFrame but want to keep doing analysis on the entire DataFrame
1. You want to select a subset of data and store it as its own variable and modify that subset without modifying your original data.

In [15]:
df1 = pd.read_csv('data/sample_data.csv', index_col=0)
df1

,state,color,food,age,height,score
Jane,NY,blue,Steak,30,165,4.6
Niko,TX,green,Lamb,2,70,8.3
Aaron,FL,red,Mango,12,120,9.0
Penelope,AL,white,Apple,4,80,3.3
Dean,AK,gray,Cheese,32,180,1.8
Christina,TX,black,Melon,33,172,9.5
Cornelia,TX,red,Beans,69,150,2.2


### No assignment!

In [17]:
df1.loc[['Aaron', 'Dean']]['color']

Aaron     red
Dean     gray
Name: color, dtype: object

In [23]:
df2 = df1.loc[['Aaron', 'Dean']]
df2

,state,color,food,age,height,score
Aaron,FL,red,Mango,12,120,9.0
Dean,AK,gray,Cheese,32,180,1.8


In [24]:
df2['color'] = 'PURPLE'
df2

,state,color,food,age,height,score
Aaron,FL,PURPLE,Mango,12,120,9.0
Dean,AK,PURPLE,Cheese,32,180,1.8


In [18]:
df1.loc[['Aaron', 'Dean']]['color'] = 'PURPLE'
df1

,state,color,food,age,height,score
Jane,NY,blue,Steak,30,165,4.6
Niko,TX,green,Lamb,2,70,8.3
Aaron,FL,red,Mango,12,120,9.0
Penelope,AL,white,Apple,4,80,3.3
Dean,AK,gray,Cheese,32,180,1.8
Christina,TX,black,Melon,33,172,9.5
Cornelia,TX,red,Beans,69,150,2.2


In [21]:
a = [0, 5]
b = a.copy()
b[0] = 99
b

[99, 5]

In [22]:
a

[0, 5]

### Idiomatic

In [25]:
rs = ['Aaron', 'Dean']
cs = 'color'
df1.loc[rs, cs] = 'PURPLE'
df1

,state,color,food,age,height,score
Jane,NY,blue,Steak,30,165,4.6
Niko,TX,green,Lamb,2,70,8.3
Aaron,FL,PURPLE,Mango,12,120,9.0
Penelope,AL,white,Apple,4,80,3.3
Dean,AK,PURPLE,Cheese,32,180,1.8
Christina,TX,black,Melon,33,172,9.5
Cornelia,TX,red,Beans,69,150,2.2


# Summary of Scenario 1:
* Use exactly one set of brackets to make the assignment
* You know you've made a mistake when you see back to back brackets like this `][`
* Separate row and column selection by a comma within the same set of brackets

# Scenario 2
Scenario 2 exists when you take a subset of data and want to keep working with just that subset. You may not care at all about the original DataFrame, but you probably won't want to change its data.

In this scenario, you will use the `copy` method to create a fresh independent copy of your subset and then make changes to that.

In [26]:
df2 = pd.read_csv('data/sample_data.csv', index_col=0)
food_score = df2[['food', 'score']]
food_score

,food,score
Jane,Steak,4.6
Niko,Lamb,8.3
Aaron,Mango,9.0
Penelope,Apple,3.3
Dean,Cheese,1.8
Christina,Melon,9.5
Cornelia,Beans,2.2


In [27]:
criteria = food_score['food'].isin(['Steak', 'Lamb'])
food_score.loc[criteria, 'score'] = 99
food_score

/Users/Ted/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


,food,score
Jane,Steak,99.0
Niko,Lamb,99.0
Aaron,Mango,9.0
Penelope,Apple,3.3
Dean,Cheese,1.8
Christina,Melon,9.5
Cornelia,Beans,2.2


In [29]:
df2

,state,color,food,age,height,score
Jane,NY,blue,Steak,30,165,4.6
Niko,TX,green,Lamb,2,70,8.3
Aaron,FL,red,Mango,12,120,9.0
Penelope,AL,white,Apple,4,80,3.3
Dean,AK,gray,Cheese,32,180,1.8
Christina,TX,black,Melon,33,172,9.5
Cornelia,TX,red,Beans,69,150,2.2


### Idiomatic
Use the `copy` method:

In [30]:
food_score = df[['food', 'score']].copy()

criteria = food_score['food'].isin(['Steak', 'Lamb'])
food_score.loc[criteria, 'score'] = 99
food_score

,food,score
Jane,Steak,99.0
Niko,Lamb,99.0
Aaron,Mango,9.0
Penelope,Apple,3.3
Dean,Cheese,1.8
Christina,Melon,9.5
Cornelia,Beans,2.2


# `.ix` is deprecated
Remove every trace of it from your code. It is ambiguous. `.loc` and `.iloc` are explicit. Use them.

### Very little reason to use `.at` and `.iat`
These two indexers select a single cell from a DataFrame/Series. There is almost never going to be a case when they are necessary. They provide a small speed-up over `.loc` and `.iloc`, but if you really wanted to select data faster then you should drop down into NumPy.

# `query` method
It is more readable but does not work with columns with spaces. It also adds no additional functionality over normal boolean indexing, so why use it?

In [31]:
df.query('age > 30')

,state,color,food,age,height,score
Dean,AK,gray,Cheese,32,180,1.8
Christina,TX,black,Melon,33,172,9.5
Cornelia,TX,red,Beans,69,150,2.2


In [33]:
df[df['age'] > 30]

,state,color,food,age,height,score
Dean,AK,gray,Cheese,32,180,1.8
Christina,TX,black,Melon,33,172,9.5
Cornelia,TX,red,Beans,69,150,2.2


In [34]:
df3 = df.copy()

In [35]:
df3 = df3.rename(columns={'food': 'fave food'})
df3

,state,color,fave food,age,height,score
Jane,NY,blue,Steak,30,165,4.6
Niko,TX,green,Lamb,2,70,8.3
Aaron,FL,red,Mango,12,120,9.0
Penelope,AL,white,Apple,4,80,3.3
Dean,AK,gray,Cheese,32,180,1.8
Christina,TX,black,Melon,33,172,9.5
Cornelia,TX,red,Beans,69,150,2.2


In [37]:
df3.query('`fave food` == "Steak"')

SyntaxError: invalid syntax (<unknown>, line 1)

# Arithmetic and Comparison Operators
Use the arithmetic and comparison operators `+, -, *, /, <, >, <=, >=, ==, !=` over their counterpart methods `add, sub, mul, div, lt, gt, le, ge, eq, ne` unless you need to change the direction of an operation.

In [38]:
college = pd.read_csv('data/college.csv', index_col='instnm')
pd.options.display.max_columns = 100
college.head()

,city,stabbr,hbcu,menonly,womenonly,relaffil,satvrmid,satmtmid,distanceonly,ugds,ugds_white,ugds_black,ugds_hisp,ugds_asian,ugds_aian,ugds_nhpi,ugds_2mor,ugds_nra,ugds_unkn,pptug_ef,curroper,pctpell,pctfloan,ug25abv,md_earn_wne_p10,grad_debt_mdn_supp
instnm,,,,,,,,,,,,,,,,,,,,,,,,,,
Alabama A & M University,Normal,AL,1.0,0.0,0.0,0,424.0,420.0,0.0,4206.0,0.0333,0.9353,0.0055,0.0019,0.0024,0.0019,0.0000,0.0059,0.0138,0.0656,1,0.7356,0.8284,0.1049,30300,33888
University of Alabama at Birmingham,Birmingham,AL,0.0,0.0,0.0,0,570.0,565.0,0.0,11383.0,0.5922,0.2600,0.0283,0.0518,0.0022,0.0007,0.0368,0.0179,0.0100,0.2607,1,0.3460,0.5214,0.2422,39700,21941.5
Amridge University,Montgomery,AL,0.0,0.0,0.0,1,NaN,NaN,1.0,291.0,0.2990,0.4192,0.0069,0.0034,0.0000,0.0000,0.0000,0.0000,0.2715,0.4536,1,0.6801,0.7795,0.8540,40100,23370
University of Alabama in Huntsville,Huntsville,AL,0.0,0.0,0.0,0,595.0,590.0,0.0,5451.0,0.6988,0.1255,0.0382,0.0376,0.0143,0.0002,0.0172,0.0332,0.0350,0.2146,1,0.3072,0.4596,0.2640,45500,24097
Alabama State University,Montgomery,AL,1.0,0.0,0.0,0,425.0,430.0,0.0,4811.0,0.0158,0.9208,0.0121,0.0019,0.0010,0.0006,0.0098,0.0243,0.0137,0.0892,1,0.7347,0.7554,0.1270,26600,33118.5


In [39]:
college_ugds = college.loc[:, 'ugds_white':'ugds_unkn']
college_ugds.head()

,ugds_white,ugds_black,ugds_hisp,ugds_asian,ugds_aian,ugds_nhpi,ugds_2mor,ugds_nra,ugds_unkn
instnm,,,,,,,,,
Alabama A & M University,0.0333,0.9353,0.0055,0.0019,0.0024,0.0019,0.0000,0.0059,0.0138
University of Alabama at Birmingham,0.5922,0.2600,0.0283,0.0518,0.0022,0.0007,0.0368,0.0179,0.0100
Amridge University,0.2990,0.4192,0.0069,0.0034,0.0000,0.0000,0.0000,0.0000,0.2715
University of Alabama in Huntsville,0.6988,0.1255,0.0382,0.0376,0.0143,0.0002,0.0172,0.0332,0.0350
Alabama State University,0.0158,0.9208,0.0121,0.0019,0.0010,0.0006,0.0098,0.0243,0.0137


In [40]:
race_ugds_mean = college_ugds.mean()
race_ugds_mean

ugds_white    0.510207
ugds_black    0.189997
ugds_hisp     0.161635
ugds_asian    0.033544
ugds_aian     0.013813
ugds_nhpi     0.004569
ugds_2mor     0.023950
ugds_nra      0.016086
ugds_unkn     0.045181
dtype: float64

### Default is to align Series index with columns

In [41]:
college_ugds_mean_diff = college_ugds - race_ugds_mean
college_ugds_mean_diff.head(10)

,ugds_white,ugds_black,ugds_hisp,ugds_asian,ugds_aian,ugds_nhpi,ugds_2mor,ugds_nra,ugds_unkn
instnm,,,,,,,,,
Alabama A & M University,-0.476907,0.745303,-0.156135,-0.031644,-0.011413,-0.002669,-0.02395,-0.010186,-0.031381
University of Alabama at Birmingham,0.081993,0.070003,-0.133335,0.018256,-0.011613,-0.003869,0.01285,0.001814,-0.035181
Amridge University,-0.211207,0.229203,-0.154735,-0.030144,-0.013813,-0.004569,-0.02395,-0.016086,0.226319
University of Alabama in Huntsville,0.188593,-0.064497,-0.123435,0.004056,0.000487,-0.004369,-0.00675,0.017114,-0.010181
Alabama State University,-0.494407,0.730803,-0.149535,-0.031644,-0.012813,-0.003969,-0.01415,0.008214,-0.031481
The University of Alabama,0.272293,-0.078097,-0.126835,-0.022944,-0.010013,-0.003669,0.00215,0.010714,-0.042581
Central Alabama Community College,0.215293,0.071303,-0.157235,-0.031044,-0.009413,-0.004569,-0.02395,-0.016086,-0.043281
Athens State University,0.272093,-0.069997,-0.142535,-0.028244,0.001887,-0.003569,-0.00655,-0.010386,-0.011781
Auburn University at Montgomery,0.022593,0.147603,-0.154235,-0.011444,-0.009413,-0.002969,0.00575,0.023614,-0.020581


In [42]:
race_school_min = college_ugds.min(axis='columns')
race_school_min.head(10)

instnm
Alabama A & M University               0.0000
University of Alabama at Birmingham    0.0007
Amridge University                     0.0000
University of Alabama in Huntsville    0.0002
Alabama State University               0.0006
The University of Alabama              0.0009
Central Alabama Community College      0.0000
Athens State University                0.0010
Auburn University at Montgomery        0.0016
Auburn University                      0.0000
dtype: float64

In [44]:
# blows up due to outer join of index
a = college_ugds - race_school_min

In [45]:
a.shape

(7535, 7544)

Arithmetic and comparison **methods** default to `axis='columns'`. Almost all others default to axis='index'. We must use the `sub` method to change the direction of operation.

In [46]:
college_ugds.sub(race_school_min, axis='index').head(10)

,ugds_white,ugds_black,ugds_hisp,ugds_asian,ugds_aian,ugds_nhpi,ugds_2mor,ugds_nra,ugds_unkn
instnm,,,,,,,,,
Alabama A & M University,0.0333,0.9353,0.0055,0.0019,0.0024,0.0019,0.0000,0.0059,0.0138
University of Alabama at Birmingham,0.5915,0.2593,0.0276,0.0511,0.0015,0.0000,0.0361,0.0172,0.0093
Amridge University,0.2990,0.4192,0.0069,0.0034,0.0000,0.0000,0.0000,0.0000,0.2715
University of Alabama in Huntsville,0.6986,0.1253,0.0380,0.0374,0.0141,0.0000,0.0170,0.0330,0.0348
Alabama State University,0.0152,0.9202,0.0115,0.0013,0.0004,0.0000,0.0092,0.0237,0.0131
The University of Alabama,0.7816,0.1110,0.0339,0.0097,0.0029,0.0000,0.0252,0.0259,0.0017
Central Alabama Community College,0.7255,0.2613,0.0044,0.0025,0.0044,0.0000,0.0000,0.0000,0.0019
Athens State University,0.7813,0.1190,0.0181,0.0043,0.0147,0.0000,0.0164,0.0047,0.0324
Auburn University at Montgomery,0.5312,0.3360,0.0058,0.0205,0.0028,0.0000,0.0281,0.0381,0.0230


# Use DataFrame/Series methods
A common mistake is to use a built-in core Python function instead of a DataFrame/Series method.

In [47]:
ugds = college['ugds'].dropna()
ugds.head(10)

instnm
Alabama A & M University                4206.0
University of Alabama at Birmingham    11383.0
Amridge University                       291.0
University of Alabama in Huntsville     5451.0
Alabama State University                4811.0
The University of Alabama              29851.0
Central Alabama Community College       1592.0
Athens State University                 2991.0
Auburn University at Montgomery         4304.0
Auburn University                      20514.0
Name: ugds, dtype: float64

In [48]:
sum(ugds)

16200904.0

In [49]:
ugds.sum()

16200904.0

## No difference except when there are missing values

In [50]:
sum(college['ugds'])

nan

In [51]:
college['ugds'].sum()

16200904.0

## Large performance difference

In [52]:
ugds1 = ugds.sample(n=10**6, replace=True)

In [57]:
import numpy as np

In [59]:
%timeit -n 5 sum(ugds1.values)

87.6 ms ± 1.54 ms per loop (mean ± std. dev. of 7 runs, 5 loops each)


In [60]:
%timeit -n 5 ugds1.values.sum()

451 µs ± 64.4 µs per loop (mean ± std. dev. of 7 runs, 5 loops each)


# `apply` - the method that does nothing but is used the most often
The `apply` method does basically nothing. It simply replaces a manual writing of a for loop.

In [61]:
college_ugds.head()

,ugds_white,ugds_black,ugds_hisp,ugds_asian,ugds_aian,ugds_nhpi,ugds_2mor,ugds_nra,ugds_unkn
instnm,,,,,,,,,
Alabama A & M University,0.0333,0.9353,0.0055,0.0019,0.0024,0.0019,0.0000,0.0059,0.0138
University of Alabama at Birmingham,0.5922,0.2600,0.0283,0.0518,0.0022,0.0007,0.0368,0.0179,0.0100
Amridge University,0.2990,0.4192,0.0069,0.0034,0.0000,0.0000,0.0000,0.0000,0.2715
University of Alabama in Huntsville,0.6988,0.1255,0.0382,0.0376,0.0143,0.0002,0.0172,0.0332,0.0350
Alabama State University,0.0158,0.9208,0.0121,0.0019,0.0010,0.0006,0.0098,0.0243,0.0137


In [62]:
college_ugds.apply(lambda x: x.max())

ugds_white    1.0000
ugds_black    1.0000
ugds_hisp     1.0000
ugds_asian    0.9727
ugds_aian     1.0000
ugds_nhpi     0.9983
ugds_2mor     0.5333
ugds_nra      0.9286
ugds_unkn     0.9027
dtype: float64

In [63]:
%timeit -n 5 college_ugds.apply(lambda x: x.max())

1.63 ms ± 149 µs per loop (mean ± std. dev. of 7 runs, 5 loops each)


In [64]:
college_ugds.max()

ugds_white    1.0000
ugds_black    1.0000
ugds_hisp     1.0000
ugds_asian    0.9727
ugds_aian     1.0000
ugds_nhpi     0.9983
ugds_2mor     0.5333
ugds_nra      0.9286
ugds_unkn     0.9027
dtype: float64

In [65]:
%timeit -n 5 college_ugds.max()

665 µs ± 108 µs per loop (mean ± std. dev. of 7 runs, 5 loops each)


In [66]:
college_ugds.apply(lambda x: x.max(), axis='columns').head()

instnm
Alabama A & M University               0.9353
University of Alabama at Birmingham    0.5922
Amridge University                     0.4192
University of Alabama in Huntsville    0.6988
Alabama State University               0.9208
dtype: float64

In [67]:
college_ugds.max(axis='columns').head()

instnm
Alabama A & M University               0.9353
University of Alabama at Birmingham    0.5922
Amridge University                     0.4192
University of Alabama in Huntsville    0.6988
Alabama State University               0.9208
dtype: float64

### Huge time difference when doing `axis='columns'`
A for-loop over the rows is a very slow operations. Avoid at all costs.

In [68]:
%timeit -n 1 -r 1 college_ugds.apply(lambda x: x.max(), axis='columns')

615 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [73]:
%timeit -n 5 college_ugds.values.max(axis=1)

162 µs ± 44.7 µs per loop (mean ± std. dev. of 7 runs, 5 loops each)


/Users/Ted/anaconda3/lib/python3.6/site-packages/numpy/core/_methods.py:28: RuntimeWarning: invalid value encountered in reduce
  return umr_maximum(a, axis, None, out, keepdims, initial)


In [69]:
%timeit -n 5 college_ugds.max(axis='columns').head()

879 µs ± 144 µs per loop (mean ± std. dev. of 7 runs, 5 loops each)


# Acceptable usages of `apply`
Only use `apply` when a built in pandas method does not exist.

In [74]:
earnings_debt = college[['md_earn_wne_p10', 'grad_debt_mdn_supp']]
earnings_debt.head()

,md_earn_wne_p10,grad_debt_mdn_supp
instnm,,
Alabama A & M University,30300,33888
University of Alabama at Birmingham,39700,21941.5
Amridge University,40100,23370
University of Alabama in Huntsville,45500,24097
Alabama State University,26600,33118.5


In [75]:
earnings_debt.dtypes

md_earn_wne_p10       object
grad_debt_mdn_supp    object
dtype: object

In [76]:
earnings_debt.astype('float')

ValueError: could not convert string to float: 'PrivacySuppressed'

In [ ]:
pd.to_numeric(earnings_debt)

In [77]:
earnings_debt.apply(pd.to_numeric, errors='coerce').head()

,md_earn_wne_p10,grad_debt_mdn_supp
instnm,,
Alabama A & M University,30300.0,33888.0
University of Alabama at Birmingham,39700.0,21941.5
Amridge University,40100.0,23370.0
University of Alabama in Huntsville,45500.0,24097.0
Alabama State University,26600.0,33118.5


# Storing complex objects inside DataFrames/Series
Just because Pandas allows you to do something, does not mean it is a good idea. There is not good support for non-scalar values stored within cells of DataFrames/Series. Store multiple values in separate columns.

In [ ]:
# never do this
college_ugds.head(20).apply(lambda x: pd.Series({'max and min': [x.min(), x.max()]}), axis=1).head()

# Know the three components of a groupby aggregation
All groupby aggregations contain 3 components:
* Grouping Columns - Unique combinations of these for independent groups
* Aggregating Columns - The values in these columns will be aggregated to a single value
* Aggregating functions - The type of aggregation to be used. Must output a single value

# `groupby` syntax - standardize for readability
There are a number of syntaxes that get used for the `groupby` method. 

In [ ]:
# syntax that I use
state_math_sat_max = college.groupby('stabbr') \
                            .agg({'satmtmid': 'max'})
state_math_sat_max.head()

In [ ]:
college.groupby('stabbr')['satmtmid'].agg('max').head()

In [ ]:
# no reason to use the full word aggregate. Always use agg
college.groupby('stabbr')['satmtmid'].aggregate('max').head()

In [ ]:
college.groupby('stabbr')['satmtmid'].max().head()

In [ ]:
college[['stabbr', 'satmtmid']].groupby('stabbr').max().head()

# Handling a MultiIndex - Usually after grouping

In [78]:
col_stats = college.groupby(['stabbr', 'relaffil']) \
                   .agg({'ugds': ['min', 'max'], 
                        'satmtmid': ['median', 'max']})
col_stats.head(10)

ugds           satmtmid       
                    min       max   median    max
stabbr relaffil                                  
AK     0          109.0   12865.0      NaN    NaN
       1           27.0     275.0    503.0  503.0
AL     0           12.0   29851.0    518.0  590.0
       1           13.0    3033.0    477.5  560.0
AR     0           18.0   21405.0    510.0  565.0
       1           20.0    4485.0    528.0  600.0
AS     0         1276.0    1276.0      NaN    NaN
AZ     0            1.0  151558.0    540.0  580.0
       1           25.0    4102.0    480.0  480.0
CA     0            0.0   44744.0    560.0  785.0

### I don't like MultiIndexes
Personally, I find that MultiIndexes add no value to pandas. Selecting subsets of data from them is not obvious. Instead, renaming the columns by hand is not a bad strategy. We can also reset the index.

In [79]:
col_stats.columns = ['min ugds', 'max ugds', 'median satmtmid', 'max satmtmid']
col_stats = col_stats.reset_index()
col_stats.head()

,stabbr,relaffil,min ugds,max ugds,median satmtmid,max satmtmid
0,AK,0,109.0,12865.0,NaN,NaN
1,AK,1,27.0,275.0,503.0,503.0
2,AL,0,12.0,29851.0,518.0,590.0
3,AL,1,13.0,3033.0,477.5,560.0
4,AR,0,18.0,21405.0,510.0,565.0


# Calling `apply` on a `groupby` object - be careful
Using `apply` within a `groupby` can lead to disastrous performance. It is one of the slowest operations in all of pandas. 

### Finding the percentage of all undergraduates represented in the top 5 most populous colleges
To accomplish this, we write a custom function to sort the values of each group from greatest to least. We then select the first 5 values with .iloc and sum them. We divide this sum by the total.

In [80]:
def top5_perc(s):
    s = s.sort_values(ascending=False)
    top5_total = s.iloc[:5].sum()
    total = s.sum()
    return top5_total / total

In [81]:
college.groupby('stabbr').agg({'ugds': top5_perc}).head(10)

,ugds
stabbr,
AK,0.961575
AL,0.370760
AR,0.422675
AS,1.000000
AZ,0.551486
CA,0.076559
CO,0.378463
CT,0.296679
DC,0.755056


# Run operations that are independent of the group outside of the custom function
The best way to avoid giant performance leaks with groupby-apply is to run all operations that are independent of the group outside of the custom aggregation function. Here, we sort the entire DataFrame first.

In [83]:
def top5_perc_simple(s):
    top5_total = s.iloc[:5].sum()
    total = s.sum()
    return top5_total / total

In [84]:
college.sort_values('ugds', ascending=False) \
       .groupby('stabbr').agg({'ugds': top5_perc_simple}).head(10)

,ugds
stabbr,
AK,0.961575
AL,0.370760
AR,0.422675
AS,1.000000
AZ,0.551486
CA,0.076559
CO,0.378463
CT,0.296679
DC,0.755056


In [85]:
%timeit -n 5 college.groupby('stabbr').agg({'ugds': top5_perc})

33.8 ms ± 2.9 ms per loop (mean ± std. dev. of 7 runs, 5 loops each)


In [86]:
%%timeit -n 5 
college.sort_values('ugds', ascending=False) \
       .groupby('stabbr').agg({'ugds': top5_perc_simple}).head(10)

17.6 ms ± 1.2 ms per loop (mean ± std. dev. of 7 runs, 5 loops each)


# Pandas Power User Optimization

In [ ]:
college_top5 = college.sort_values('ugds', ascending=False) \
                      .groupby('stabbr').head()

In [ ]:
top5_total = college_top5.groupby('stabbr').agg({'ugds': 'sum'})
top5_total.head()

In [ ]:
total = college.groupby('stabbr').agg({'ugds': 'sum'})
total.head()

In [ ]:
(top5_total / total).head()

In [ ]:
%%timeit -n 5
college_top5 = college.sort_values('ugds', ascending=False) \
                      .groupby('stabbr').head()
top5_total = college_top5.groupby('stabbr').agg({'ugds': 'sum'})
total = college.groupby('stabbr').agg({'ugds': 'sum'})
top5_total / total

# `melt` vs `stack`
These methods are virtually identical. I prefer `melt` as it avoids a multi-level index.

In [87]:
movie = pd.read_csv('data/movie.csv')
movie.head()

,title,actor1,actor1_fb,actor2,actor2_fb,actor3,actor3_fb
0,Avatar,CCH Pounder,1000.0,Joel David Moore,936.0,Wes Studi,855.0
1,Pirates of the Caribbean: At World's End,Johnny Depp,40000.0,Orlando Bloom,5000.0,Jack Davenport,1000.0
2,Spectre,Christoph Waltz,11000.0,Rory Kinnear,393.0,Stephanie Sigman,161.0
3,The Dark Knight Rises,Tom Hardy,27000.0,Christian Bale,23000.0,Joseph Gordon-Levitt,23000.0
4,Star Wars: Episode VII - The Force Awakens,Doug Walker,131.0,Rob Walker,12.0,NaN,NaN


In [88]:
act1 = movie.melt(id_vars=['title'], 
                  value_vars=['actor1', 'actor2', 'actor3'], 
                  var_name='actor number',
                  value_name='actor name')

In [89]:
stacked = movie.set_index('title')[['actor1', 'actor2', 'actor3']].stack()
stacked.head()

title                                           
Avatar                                    actor1         CCH Pounder
                                          actor2    Joel David Moore
                                          actor3           Wes Studi
Pirates of the Caribbean: At World's End  actor1         Johnny Depp
                                          actor2       Orlando Bloom
dtype: object

In [90]:
stacked.reset_index(name='actor name').head(10)

,title,level_1,actor name
0,Avatar,actor1,CCH Pounder
1,Avatar,actor2,Joel David Moore
2,Avatar,actor3,Wes Studi
3,Pirates of the Caribbean: At World's End,actor1,Johnny Depp
4,Pirates of the Caribbean: At World's End,actor2,Orlando Bloom
5,Pirates of the Caribbean: At World's End,actor3,Jack Davenport
6,Spectre,actor1,Christoph Waltz
7,Spectre,actor2,Rory Kinnear
8,Spectre,actor3,Stephanie Sigman
9,The Dark Knight Rises,actor1,Tom Hardy


In [91]:
act1.pivot(index='title', columns='actor number', values='actor name').head()

actor number,actor1,actor2,actor3
title,,,
#Horror,Timothy Hutton,Balthazar Getty,Lydia Hearst
10 Cloverfield Lane,Bradley Cooper,John Gallagher Jr.,Sumalee Montano
10 Days in a Madhouse,Christopher Lambert,Kelly LeBrock,Alexandra Callas
10 Things I Hate About You,Joseph Gordon-Levitt,Heath Ledger,Andrew Keegan
"10,000 B.C.",Mathew Buck,NaN,NaN


In [92]:
stacked.unstack().head()

,actor1,actor2,actor3
title,,,
Avatar,CCH Pounder,Joel David Moore,Wes Studi
Pirates of the Caribbean: At World's End,Johnny Depp,Orlando Bloom,Jack Davenport
Spectre,Christoph Waltz,Rory Kinnear,Stephanie Sigman
The Dark Knight Rises,Tom Hardy,Christian Bale,Joseph Gordon-Levitt
Star Wars: Episode VII - The Force Awakens,Doug Walker,Rob Walker,NaN


# `pivot_table` vs `groupby` then `unstack`
`pivot_table` can directly create a pivot table. You can achieve the exact same result by grouping by multiple columns and then unstacking. I prefer the pivot table as it is clearer.

In [ ]:
emp = pd.read_csv('data/employee.csv')
emp.head()

In [ ]:
emp.pivot_table(index='race', columns='gender', values='salary')

In [ ]:
race_gen_sal = emp.groupby(['race', 'gender']).agg({'salary': 'mean'})
race_gen_sal

In [ ]:
race_gen_sal.unstack('gender')